In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,67805
2,Huelva,Confirmados PDIA 14 días,7891
3,Huelva,Tasa PDIA 14 días,"1537,697059453982"
4,Huelva,Confirmados PDIA 7 días,3509
5,Huelva,Tasa PDIA 7 dias,"683,7889977980007"
6,Huelva,Total Confirmados,68031
7,Huelva,Curados,52275
8,Huelva,Fallecidos,432


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  67805.0


/tmp/ipykernel_56069/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  19816.0


/tmp/ipykernel_56069/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_56069/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_56069/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_56069/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 132 personas en los últimos 7 días 

Un positivo PCR cada 56 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,67805.0,7891.0,3509.0,513170.0,683.788998,1537.697059,982.0
Huelva-Costa,39581.0,4351.0,1851.0,289548.0,639.272245,1502.686947,477.0
Sierra de Huelva-Andévalo Central,8094.0,1794.0,855.0,67391.0,1268.715407,2662.076538,309.0
Huelva (capital),19816.0,2577.0,1089.0,143837.0,757.107003,1791.611338,250.0
Condado-Campiña,19670.0,1736.0,801.0,156231.0,512.702345,1111.175119,196.0
Valverde del Camino,1787.0,367.0,167.0,12750.0,1309.803922,2878.431373,78.0
Aljaraque,2691.0,380.0,156.0,21474.0,726.459905,1769.581820,51.0
Nerva,551.0,187.0,94.0,5169.0,1818.533565,3617.721029,40.0
Cortegana,591.0,155.0,101.0,4602.0,2194.697957,3368.100826,39.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,30.0,18.0,2.0,330.0,606.060606,5454.545455,2.0
Cerro de Andévalo (El),378.0,124.0,62.0,2327.0,2664.374731,5328.749463,15.0
Rosal de la Frontera,205.0,75.0,33.0,1697.0,1944.608132,4419.563936,8.0
Fuenteheridos,75.0,26.0,14.0,709.0,1974.612130,3667.136812,6.0
Nerva,551.0,187.0,94.0,5169.0,1818.533565,3617.721029,40.0
Cumbres Mayores,283.0,63.0,41.0,1749.0,2344.196684,3602.058319,14.0
Cortegana,591.0,155.0,101.0,4602.0,2194.697957,3368.100826,39.0
Zalamea la Real,293.0,102.0,41.0,3054.0,1342.501637,3339.882122,14.0
Jabugo,230.0,75.0,39.0,2260.0,1725.663717,3318.584071,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Encinasola,198.0,32.0,3.0,1310.0,229.007634,2442.748092,0.0,0.093750
Hinojales,30.0,18.0,2.0,330.0,606.060606,5454.545455,2.0,0.111111
Villalba del Alcor,562.0,23.0,4.0,3366.0,118.835413,683.303624,0.0,0.173913
Cabezas Rubias,64.0,9.0,2.0,706.0,283.286119,1274.787535,0.0,0.222222
Almonaster la Real,183.0,41.0,10.0,1788.0,559.284116,2293.064877,0.0,0.243902
Galaroza,126.0,16.0,4.0,1382.0,289.435601,1157.742402,1.0,0.250000
Corteconcepción,72.0,8.0,2.0,536.0,373.134328,1492.537313,1.0,0.250000
San Silvestre de Guzmán,83.0,4.0,1.0,614.0,162.866450,651.465798,1.0,0.250000
Higuera de la Sierra,119.0,19.0,5.0,1291.0,387.296669,1471.727343,0.0,0.263158
